# Plot clustermatch results

The input to clustermatch was k-mer abundances that had been MR-normalized, logged, and filtered to k-mers that were present in at least 50 samples.

In [1]:
setwd("..")

In [2]:
# change default figure size
options(repr.plot.width=15, repr.plot.height=7)
# disable scientific notation (for plot axes)
options(scipen = 999)

In [4]:
library(readr)
library(dplyr)
library(tibble)
library(tidyr)
library(purrr)
library(ggplot2)

## Functions

In [5]:
# transform the square correlation matrix into long format
cor_to_long_cm <- function(m){
    #df <- m %>%
    #  as.data.frame %>% 
    #  tibble::rownames_to_column() %>% 
    #  tidyr::pivot_longer(-rowname) %>%
    #  select(row = rowname, col = name, corr = value) %>%
    #  filter(row != col)
    m <- m %>%
      column_to_rownames("minhash")
    df <- data.frame(row=rownames(m)[row(m)], col=colnames(m)[col(m)], corr=c(m)) %>%
      filter(row != col)
    df <- df %>% 
      mutate(row = as.numeric(row)) %>% # convery minhash back to numeric to perserve joins later
      mutate(col = as.numeric(row))
} 

In [6]:
clustermatch <- read_tsv("~/Downloads/cm-pa14_mrnorm_hlog_kmers_filt50.tsv", show_col_types = F)

In [7]:
clustermatch[1:5, 1:5]

minhash,6420423175363,7037975305246,8687411883152,12877586329792
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
6420423175363,1.00000,0.33252,0.33426,0.43772
7037975305246,0.33252,1.00000,0.47444,0.20970
8687411883152,0.33426,0.47444,1.00000,0.32522
12877586329792,0.43772,0.20970,0.32522,1.00000
18650490225240,0.43772,0.31049,0.37990,0.43772


## Read in hash map and label with operons

In [8]:
pa14_hash_map <- read_csv("outputs/txomes_sourmash_sketch_singleton/pa14.csv", show_col_types = F)

In [9]:
pa14_operons <- read_csv("https://raw.githubusercontent.com/greenelab/core-accessory-interactome/master/data/metadata/PA14-operons-2021-07-19.csv", show_col_types = F)

In [10]:
pa14_transcript_to_gene <- read_csv("https://osf.io/ema5c/download", skip = 1, col_names = c("tmp", "transcript_name", "locus_tag"), show_col_types = F) %>%
  select(-tmp)

In [11]:
pa14_hash_map <- pa14_hash_map %>%
  mutate(transcript_name = gsub(" .*", "", name)) %>%
  left_join(pa14_transcript_to_gene, by = "transcript_name") %>%
  left_join(pa14_operons, by = "locus_tag")

In [12]:
pa14_hash_map_operon <- pa14_hash_map %>%
  filter(!is.na(`operon-id`))

pa14_hash_map_operon_small <- pa14_hash_map_operon %>%
  select(minhash, operon_id = `operon-id`) %>%
  distinct()

## Plot results

In [ ]:
# add operon metadata and plot
clustermatch_df <- cor_to_long_cm(clustermatch) %>%
  left_join(pa14_hash_map_operon_small, by = c("row" = "minhash")) %>% # determine which operon, if any, the row is part of
  select(row, col, corr, operon_id_row = operon_id) %>% # record operon identity of row
  left_join(pa14_hash_map_operon_small, by = c("col" = "minhash")) %>% # determine which operon the col is part of
  select(row, col, corr, operon_id_row, operon_id_col = operon_id) %>% # record operon identity of col
  mutate(operon = ifelse(operon_id_row == operon_id_col, "operon pair", "not operon pair")) %>% # label operon pairs
  mutate(operon = ifelse(is.na(operon), "not operon pair", operon))

ggplot(clustermatch_df, aes(x = corr, fill = operon)) +
  geom_histogram(alpha = .5) +
  theme_classic()